# Circuit autointerpretability

This stuff just sets up everything we need.

In [43]:
from autointerpretability import *

# Autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
config = yaml.safe_load(open("config.yaml"))
llm_client = AzureOpenAI(
    azure_endpoint=config["base_url"],
    api_key=config["azure_api_key"],
    api_version=config["api_version"],
)

model = HookedTransformer.from_pretrained('gpt2-small')

dataset = load_dataset('Skylion007/openwebtext', split='train', streaming=True)
dataset = dataset.shuffle(seed=42, buffer_size=10_000)
tokenized_owt = tokenize_and_concatenate(dataset, model.tokenizer, max_length=128, streaming=True)
tokenized_owt = tokenized_owt.shuffle(42)
tokenized_owt = tokenized_owt.take(12800 * 2)
owt_tokens = np.stack([x['tokens'] for x in tokenized_owt])
owt_tokens_torch = torch.tensor(owt_tokens)

device = 'cpu'
tl_model, z_saes, transcoders = get_model_encoders(device=device)

Loaded pretrained model gpt2-small into HookedTransformer


/Users/charlesoneill/miniconda3/envs/anu/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Token indices sequence length is longer than the specified maximum sequence length for this model (73252 > 1024). Running this sequence through 

Loaded pretrained model gpt2-small into HookedTransformer

Loading SAEs...


100%|██████████| 12/12 [00:08<00:00,  1.42it/s]



Loading Transcoders...


100%|██████████| 12/12 [00:04<00:00,  2.41it/s]


Note you can specify the features you want to examine, in each layer, and just pass in either the relevant ZSAE or MLP transcoder depending on what component you want to look at. The `get_feature_scores` function will handle the differences. Let's have a look at the max-activating examples on Danny's features he wanted to check out (note you can slice `owt_tokens_torch` to run for shorter).

In [11]:
features = [16513, 7861]
sae = z_saes[8]
feature_scores = get_feature_scores(model, sae, owt_tokens_torch, features, batch_size=128)

ZSAE


100%|██████████| 200/200 [04:08<00:00,  1.24s/it]


Our feature scores are a tensor of shape `(batch, feature, seq_pos)`, and so I've got a function to help extract the max-activating examples for each feature. You need to specify the feature index, which is why it's helpful to know from above the features in your list.

In [8]:
feature_idx = 0 # corresponding to 16513
example_html, examples_clean_text = display_top_k_activating_examples(model, feature_scores[:, feature_idx, :], owt_tokens_torch, k=5, show_score=True)

In [9]:
top_tokens, top_logits = get_top_k_tokens(model, sae, features[feature_idx], k=10, act_strength=3)

pretty_print_tokens_logits(top_tokens, top_logits)

╒═════════════╤═════════╕
│ Token       │   Logit │
╞═════════════╪═════════╡
│ arth        │  3.6752 │
├─────────────┼─────────┤
│ rers        │  3.4801 │
├─────────────┼─────────┤
│ displayText │  3.3468 │
├─────────────┼─────────┤
│ pool        │  3.3323 │
├─────────────┼─────────┤
│ rovers      │  3.2823 │
├─────────────┼─────────┤
│ qua         │  3.28   │
├─────────────┼─────────┤
│ assian      │  3.2042 │
├─────────────┼─────────┤
│ cember      │  3.1544 │
├─────────────┼─────────┤
│ rer         │  3.1482 │
├─────────────┼─────────┤
│ iple        │  3.14   │
╘═════════════╧═════════╛


You can also pass in and boost logits for multiple features at a time.

In [27]:
top_tokens, top_logits = get_top_k_tokens(model, sae, features, k=10, act_strength=1.5)

pretty_print_tokens_logits(top_tokens, top_logits)

╒════════════╤═════════╕
│ Token      │   Logit │
╞════════════╪═════════╡
│ rers       │  2.9971 │
├────────────┼─────────┤
│ pool       │  2.9937 │
├────────────┼─────────┤
│ ulk        │  2.8326 │
├────────────┼─────────┤
│ legate     │  2.8219 │
├────────────┼─────────┤
│ sembly     │  2.81   │
├────────────┼─────────┤
│ forum      │  2.7986 │
├────────────┼─────────┤
│  festivals │  2.765  │
├────────────┼─────────┤
│ arth       │  2.7315 │
├────────────┼─────────┤
│ cember     │  2.6992 │
├────────────┼─────────┤
│  newsp     │  2.6667 │
╘════════════╧═════════╛


Then, you can just pass it off to GPT-4 to interpret what's going on. Note that I haven't got access to `GPT-4o` with my credits yet, so this will have to wait a few days.

In [38]:
feature_interpretation = get_response(llm_client, examples_clean_text, top_tokens)

In [40]:
print(feature_interpretation)

(Part 1)
Step 1.
ACTIVATING TOKENS: "in the county", "days", "asia", "half, and", "ial,", "Byndom", "ia,", "plate", "to", "resett".
PREVIOUS TOKENS: "evacuated in", "-", "un", "par", "cliq", "Carr", "Pers", "name", "res", "charact".

Step 2.
The activating tokens are a mixture of prepositions, conjunctions, parts of words, days of the week and multipart words. 
The previous tokens have nothing in common.

Step 3.
- Many activating tokens are parts of words or phrases.
- The texts geographically widespread places.

(Part 2)
Step 4.
SIMILAR TOKENS: "arth", "rers", "rovers", "rer".
These tokens seem to be part of words, particularly endings part of nouns, adjectives or even verbs. 

Step 5:
[EXPLANATION]: Parts of words, notably the endings of nouns, verbs, or adjectives.


Finally, we can pass in multiple features at once to see the max activating examples for features together.

In [44]:
_ = display_top_k_activating_examples_sum(model, feature_scores, owt_tokens, [0, 1], k=5, show_score=True)

However, instead of passing in individual features for specific components in specific layers, I created an object called `CircuitPrediction` to basically store all this stuff for you. I'll quickly illustrate how to use it in conjunction with the above.

In [ ]:
cp = get_circuit_prediction(task='ioi', N=50)

The main thing you'll want to do with this is get features from certain components to look at on a specific task. The features for each component are stored in the circuit hypergraph. For instance:

In [ ]:
cp.circuit_hypergraph

If you want to look at MLP 3, all you have to do is access it:

In [ ]:
cp.circuit_hypergraph['MLP3']

And just repeat what we did above:

In [ ]:
features = list(set(cp.circuit_hypergraph['MLP3']['features']))
transcoder = transcoders[3]
feature_scores = get_feature_scores(model, transcoder, owt_tokens_torch, features, batch_size=64)

In [ ]:
feature_idx = 0 # corresponding to 16513
example_html, examples_clean_text = display_top_k_activating_examples(model, feature_scores[:, 0, :], owt_tokens_torch, k=5, show_score=True)

There's a few other methods, but you probably don't need to bother with those.

In [ ]:
_ = cp.unique_feature_array(visualize=True)

## Top logits from features